In [ ]:
%matplotlib inline

In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam,RMSprop
from keras import backend as K
from skimage.io import imread
from datetime import datetime
import time


In [ ]:
import matplotlib.pyplot as plt

import sys

import numpy as np
import pandas as pd
import copy
import tensorflow as tf

import os
import re
import cv2
from glob import glob
from PIL import Image
from datetime import datetime as dt
from time import time
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
plt.rcParams["axes.grid"] = False

# load_data

In [ ]:
df = pd.DataFrame(columns=['path', 'angle', 'pos', 'datetime', 'cc', 'row', 'col', 'label'])
files = sorted(glob("img2/*/*/*/*/*"))
for f in files:
    f_name = f.split("CAM_")[1].split("_")
    df.loc[len(df)] = [f, f_name[0], f_name[1], "-".join(f_name[2:5]),
                       f_name[5][5:], f_name[6][3:], f_name[7][3:-1], f[5:7]]
    
df['datetime_id'] = df['datetime']
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y%m%d-%H%M%S-%f")
df['cc'].replace('', np.NaN, inplace=True)
for i in range(0,len(df)):
    df['datetime_id'][i]= df['datetime_id'][i].replace("-","")

In [ ]:
df.tail()

In [ ]:
def input_preproc(angle='Z', label='NG'):
    sqr_img = []
    path_list = []
    for idx, path in enumerate(df[(df['angle']==angle) & (df['label']==label)]['path']):
        img = cv2.imread(path)
    
        # convert img to square
        height, width, _ = img.shape
        size = max(height, width)
        empty_img = np.zeros((size, size, 3), np.uint8)

        x_offset = int((size - width) / 2)
        y_offset = int((size - height) / 2)
        empty_img[y_offset : y_offset + height, x_offset : x_offset + width] = img

        empty_img = cv2.resize(empty_img, (size, size), interpolation=cv2.INTER_CUBIC)
        sqr_img.append(cv2.cvtColor(empty_img, cv2.COLOR_BGR2RGB))
        path_list.append(path)
        
    return sqr_img, path_list

In [ ]:
up_ng, path_list_up_ng = input_preproc(angle='UP', label='NG')
up_ok, path_list_up_ok = input_preproc(angle='UP', label='OK')

# detection_y

In [ ]:
import numpy as np
import cv2
import operator

def detection_y(data,path_list):
    cnt_1 = 0
    img_detection = np.empty((0, 100, 200, 3), dtype='uint8') # empty array to append img
    ##for i in up_ok[1400:1410] pos3
    for i in data:
        img_color = copy.deepcopy(i)
        string_path = path_list[cnt_1]

        #input_data = cv2.fastNlMeansDenoisingColored(img_color.copy(),None,7,7,7,13)
        # BGR(파랑색)
        #color = [255, 0, 0]
#         color = [50, 255, 255]
#         pixel = np.uint8([[color]])
#         # cvtColor를 사용하여 HSV 색공간으로 변환한다.
#         hsv = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
#         # HSV값을 출력하기위해 픽셀값만 갖고온다.
#         hsv = hsv[0][0]

        # bgr과 hsv 값 출력
        #print("bgr: ", color)
        #print("hsv: ", hsv)

        # 이미지파일을 컬러로 읽어온다.
        #img_color = cv2.imread('food.jpg')

        # 이미지의 높이와 너비를 갖고온다.
        height,width = img_color.shape[:2]
        shape = img_color.shape[0]
        #print(shape)

        # hsv이미지로 변환한다.
        img_hsv = cv2.cvtColor(img_color.copy(), cv2.COLOR_BGR2HSV)

        # 범위를 정하여 hsv이미지에서 원하는 색 영역을 바이너리 이미지로 생성한다.
        # (가운데와 마지막값은 너무어두워서 검은색에 가까운색과 너무 옅어서 흰색에 가까운색을 제외시키기위해 30으로해야한다.
        #lower_blue = (120-10, 30, 30)
        #upper_blue = (120+10, 255, 255)
        # yellow color upper and lower bound   

        # 앞서 선언한 범위값을 사용하여 바이너리 이미지를 얻는다.(범위내에 있는 픽셀들은 흰색이되고 나머지는 검은색이 된다.)
        mask1 = cv2.inRange(img_hsv.copy(), (60,60,60), (180, 200, 200))
        kernel = np.ones((2,2),np.uint8)        
        dilation = cv2.dilate(mask1.copy(),kernel,iterations = 1)
        opening = cv2.morphologyEx(dilation.copy(), cv2.MORPH_OPEN, kernel)
        #closing = cv2.morphologyEx(opening.copy(), cv2.MORPH_CLOSE, kernel)
        _, thresh = cv2.threshold(opening, 60, 255, cv2.THRESH_BINARY)


        bluecnts2 = cv2.findContours(thresh,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

        img_color2 = copy.deepcopy(img_color)

        list_val = []        
        for cnt in bluecnts2:                        
            x2, y2, w2, h2 = cv2.boundingRect(cnt)
            area_x = cv2.contourArea(cnt)
            rect2 = cv2.rectangle(img_color2, (x2, y2), (x2 + w2, y2 + h2), (0,255,0), 1)        
            list_val.append(area_x)

        index1, value1 = max(enumerate(list_val), key=operator.itemgetter(1))
        index2 = []
        ex_list = [[],[]]
        count_num = 0
        #print('contour length = ', len(bluecnts2))
        for cnt in bluecnts2:                                
            area_x = cv2.contourArea(cnt)
            if (value1/4) < area_x:#  or (value1/) < area_x:
                index2.append(count_num)            
                ex_list[0].append(cnt)            
                ex_list[1].append(area_x)            
            count_num = count_num + 1

        img_color3 = copy.deepcopy(img_color)
        img_color4 = copy.deepcopy(img_color)
        #print('counts of rectangle = ',index2)
        #index2.sort(reverse=True)
        #print('[0] = ',ex_list[0])
        #print('[1] = ',ex_list[1])
        if len(index2) > 1 :
            for cnt in index2:                        
                x3, y3, w3, h3 = cv2.boundingRect(bluecnts2[cnt])        
                rect3 = cv2.rectangle(img_color3, (x3, y3), (x3 + w3, y3 + h3), (0,255,0), 1)

            x5, y5, w5, h5 = cv2.boundingRect(bluecnts2[index2[0]])        
            x6, y6, w6, h6 = cv2.boundingRect(bluecnts2[index2[1]])
            x7 = 0 
            y7 = 0
            w7 = 0
            h7 =0
            x8 = 0 
            y8 = 0
            if x5 > x6:
                x7 = x6
            else:
                x7 = x5

            if y5 < y6:
                y7 = y5
            else:
                y7 = y6


            if x5 + w5 < x6 + w6:
                x8 = x6
                w7 = w6        
            else:
                x8 = x5
                w7 = w5

            if y5 + h5 < y6 + h6:
                y8 = y6
                h7 = h6

            else:
                y8 = y5
                h7 = h5

            rect4 = cv2.rectangle(img_color4, (x7-10, y7-20), (x8 + w7+10, y8 + h7+10), (0,255,0), 2)
#             fontface = cv2.FONT_HERSHEY_SIMPLEX
#             scale = 0.6
#             thickness = 2
#             size = cv2.getTextSize(string_path, fontface, scale, thickness)
#             text_width = size[0][0]
#             text_height = size[0][1]
#             pt = (x7 + int((w7 - text_width) / 2), y7 + int((h7 + text_height) / 2))
#             cv2.putText(img_color, string_path, pt, fontface, scale, (255, 255, 255), thickness, 8)
            img_resized_1 = img_color[y7-20:y8 + h7+10, x7-10: x8 + w7+10] # crop
            try:
                img_resized_2 = cv2.resize(img_resized_1.copy(), (200, 100), interpolation=cv2.INTER_CUBIC)
                cnt_1 = cnt_1 + 1 
            except:
                img_resized_1 = img_color[y7:y8 + h7, x7: x8 + w7] # crop
                img_resized_2 = cv2.resize(img_resized_1.copy(), (200, 100), interpolation=cv2.INTER_CUBIC)
                cnt_1 = cnt_1 + 1 
                

#             except:
#                 img_re = img_resized(img_color.copy(),x7,y7,w7,h7)
#                 img_resized_2 = cv2.resize(img_re.copy(), (200, 100), interpolation=cv2.INTER_CUBIC)
#                 print('*'*135)
#                 plt.imshow(img_resized_2)
#                 plt.show()
#                 print('*'*135)
            


                



        else:
            arr = []
            for cnt in index2:                        
                x3, y3, w3, h3 = cv2.boundingRect(bluecnts2[cnt])
                #if (shape/2) > (x3 + w3 + 10):                    
                rect4 = cv2.rectangle(img_color4, (x3-10, y3-20), (x3 + w3 + 10, y3 + h3 + 20), (0,255,0), 2)    
                img_resized_1 = img_color[y3-20:y3 + h3 + 20, x3-10:x3 + w3 + 10] # crop
                img_resized_2 = cv2.resize(img_resized_1.copy(), (200, 100), interpolation=cv2.INTER_CUBIC)
                x2, y2, w2, h2 = cv2.boundingRect(bluecnts2[cnt])
                rect2 = cv2.rectangle(img_color2.copy(), (x2, y2), (x2 + w2, y2 + h2), (0,255,0), 1)
                print(cnt_1)
                cnt_1 = cnt_1 + 1 
                print(string_path)
                #print(bluecnts2[cnt])
                #### convert contour to rectangles
                xywh = []
                for cnt1 in bluecnts2[cnt]:
                # ___________________________________________________________________________________ 컨투어 사각형 좌표 리턴
                    _x, _y, _w, _h = np.array(cv2.boundingRect(cnt1) + np.array([-5,-10,10,20])) # add padding to rectangles
                # ___________________________________________________________________________________
                    xywh.append(list(map(int, [_x, _y, _w, _h])))
                # _______________________________________________________________________________________ 중복 사각형 병합
                rect, _ = cv2.groupRectangles(xywh+xywh, 0, 0.001) # merge contours with intersection
                # _______________________________________________________________________________________
                print('o'*135)
                plt.figure(figsize=(30,10))    
                plt.subplot(1,4,1)
                plt.imshow(rect2)
                plt.subplot(1,4,2)
                plt.imshow(rect4)
                plt.subplot(1,4,3)
                plt.imshow(rect)
                plt.subplot(1,4,4)
                plt.imshow(img_resized_2)
                plt.show()
                print('o'*135)

        img_detection = np.concatenate((img_detection, img_resized_2[np.newaxis,...]), axis=0)        
        print(cnt_1)
        temp = string_path[0:-4].split('CAM_')        
#        cv2.imwrite('img//UP_OK_2/{}.jpg'.format(temp[1]), cv2.cvtColor(img_resized_2, cv2.COLOR_RGB2BGR))        
        print(string_path)
#         plt.figure(figsize=(30,10))    
#         plt.subplot(1,4,1)
#         plt.imshow(img_color)
#         plt.subplot(1,4,2)
#         plt.imshow(rect2)
#         plt.subplot(1,4,3)
#         plt.imshow(rect3)
#         plt.subplot(1,4,4)
#         plt.imshow(rect4) 
#         plt.subplot(1,6,5)
#         plt.imshow(dilation)
#         plt.subplot(1,6,6)
#         plt.imshow(img_resized_2) 
        plt.show()
        print('='*135)
        

    return img_detection


In [ ]:
import numpy as np
import cv2
import operator

def detection_y_1(data,path_list):
    cnt_1 = 0
    img_detection = np.empty((0, 48, 100, 3), dtype='uint8') # empty array to append img
    ##for i in up_ok[1400:1410] pos3
    for i in data:
        img_color = copy.deepcopy(i)
        string_path = path_list[cnt_1]

        #input_data = cv2.fastNlMeansDenoisingColored(img_color.copy(),None,7,7,7,13)
        # BGR(파랑색)
        #color = [255, 0, 0]
#         color = [50, 255, 255]
#         pixel = np.uint8([[color]])
#         # cvtColor를 사용하여 HSV 색공간으로 변환한다.
#         hsv = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
#         # HSV값을 출력하기위해 픽셀값만 갖고온다.
#         hsv = hsv[0][0]

        # bgr과 hsv 값 출력
        #print("bgr: ", color)
        #print("hsv: ", hsv)

        # 이미지파일을 컬러로 읽어온다.
        #img_color = cv2.imread('food.jpg')

        # 이미지의 높이와 너비를 갖고온다.
        height,width = img_color.shape[:2]
        shape = img_color.shape[0]
        #print(shape)

        # hsv이미지로 변환한다.
        img_hsv = cv2.cvtColor(img_color.copy(), cv2.COLOR_BGR2HSV)

        # 범위를 정하여 hsv이미지에서 원하는 색 영역을 바이너리 이미지로 생성한다.
        # (가운데와 마지막값은 너무어두워서 검은색에 가까운색과 너무 옅어서 흰색에 가까운색을 제외시키기위해 30으로해야한다.
        #lower_blue = (120-10, 30, 30)
        #upper_blue = (120+10, 255, 255)
        # yellow color upper and lower bound   

        # 앞서 선언한 범위값을 사용하여 바이너리 이미지를 얻는다.(범위내에 있는 픽셀들은 흰색이되고 나머지는 검은색이 된다.)
        mask1 = cv2.inRange(img_hsv.copy(), (60,60,60), (180, 200, 200))
        kernel = np.ones((2,2),np.uint8)        
        dilation = cv2.dilate(mask1.copy(),kernel,iterations = 1)
        opening = cv2.morphologyEx(dilation.copy(), cv2.MORPH_OPEN, kernel)
        #closing = cv2.morphologyEx(opening.copy(), cv2.MORPH_CLOSE, kernel)
        _, thresh = cv2.threshold(opening, 60, 255, cv2.THRESH_BINARY)


        bluecnts2 = cv2.findContours(thresh,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

        img_color2 = copy.deepcopy(img_color)

        list_val = []        
        for cnt in bluecnts2:                        
            x2, y2, w2, h2 = cv2.boundingRect(cnt)
            area_x = cv2.contourArea(cnt)
            rect2 = cv2.rectangle(img_color2, (x2, y2), (x2 + w2, y2 + h2), (0,255,0), 1)        
            list_val.append(area_x)

        index1, value1 = max(enumerate(list_val), key=operator.itemgetter(1))
        index2 = []
        ex_list = [[],[]]
        count_num = 0
        #print('contour length = ', len(bluecnts2))
        for cnt in bluecnts2:                                
            area_x = cv2.contourArea(cnt)
            if (value1/4) < area_x:#  or (value1/) < area_x:
                index2.append(count_num)            
                ex_list[0].append(cnt)            
                ex_list[1].append(area_x)            
            count_num = count_num + 1

        img_color3 = copy.deepcopy(img_color)
        img_color4 = copy.deepcopy(img_color)
        #print('counts of rectangle = ',index2)
        #index2.sort(reverse=True)
        #print('[0] = ',ex_list[0])
        #print('[1] = ',ex_list[1])
        if len(index2) > 1 :
            for cnt in index2:                        
                x3, y3, w3, h3 = cv2.boundingRect(bluecnts2[cnt])        
                rect3 = cv2.rectangle(img_color3, (x3, y3), (x3 + w3, y3 + h3), (0,255,0), 1)

            x5, y5, w5, h5 = cv2.boundingRect(bluecnts2[index2[0]])        
            x6, y6, w6, h6 = cv2.boundingRect(bluecnts2[index2[1]])
            x7 = 0 
            y7 = 0
            w7 = 0
            h7 =0
            x8 = 0 
            y8 = 0
            if x5 > x6:
                x7 = x6
            else:
                x7 = x5

            if y5 < y6:
                y7 = y5
            else:
                y7 = y6


            if x5 + w5 < x6 + w6:
                x8 = x6
                w7 = w6        
            else:
                x8 = x5
                w7 = w5

            if y5 + h5 < y6 + h6:
                y8 = y6
                h7 = h6

            else:
                y8 = y5
                h7 = h5

            rect4 = cv2.rectangle(img_color4, (x7-10, y7-20), (x8 + w7+10, y8 + h7+10), (0,255,0), 2)
#             fontface = cv2.FONT_HERSHEY_SIMPLEX
#             scale = 0.6
#             thickness = 2
#             size = cv2.getTextSize(string_path, fontface, scale, thickness)
#             text_width = size[0][0]
#             text_height = size[0][1]
#             pt = (x7 + int((w7 - text_width) / 2), y7 + int((h7 + text_height) / 2))
#             cv2.putText(img_color, string_path, pt, fontface, scale, (255, 255, 255), thickness, 8)
            img_resized_1 = img_color[y7-20:y8 + h7+10, x7-10: x8 + w7+10] # crop
            try:
                img_resized_2 = cv2.resize(img_resized_1.copy(), (100, 48), interpolation=cv2.INTER_CUBIC)
                cnt_1 = cnt_1 + 1 
            except:
                img_resized_1 = img_color[y7:y8 + h7, x7: x8 + w7] # crop
                img_resized_2 = cv2.resize(img_resized_1.copy(), (100, 48), interpolation=cv2.INTER_CUBIC)
                cnt_1 = cnt_1 + 1 
                

#             except:
#                 img_re = img_resized(img_color.copy(),x7,y7,w7,h7)
#                 img_resized_2 = cv2.resize(img_re.copy(), (200, 100), interpolation=cv2.INTER_CUBIC)
#                 print('*'*135)
#                 plt.imshow(img_resized_2)
#                 plt.show()
#                 print('*'*135)
            


                



        else:
            arr = []
            for cnt in index2:                        
                x3, y3, w3, h3 = cv2.boundingRect(bluecnts2[cnt])
                #if (shape/2) > (x3 + w3 + 10):                    
                rect4 = cv2.rectangle(img_color4, (x3-10, y3-20), (x3 + w3 + 10, y3 + h3 + 20), (0,255,0), 2)    
                img_resized_1 = img_color[y3-20:y3 + h3 + 20, x3-10:x3 + w3 + 10] # crop
                img_resized_2 = cv2.resize(img_resized_1.copy(), (100, 48), interpolation=cv2.INTER_CUBIC)
                x2, y2, w2, h2 = cv2.boundingRect(bluecnts2[cnt])
                rect2 = cv2.rectangle(img_color2.copy(), (x2, y2), (x2 + w2, y2 + h2), (0,255,0), 1)
                print(cnt_1)
                cnt_1 = cnt_1 + 1 
                print(string_path)
                #print(bluecnts2[cnt])
                #### convert contour to rectangles
                xywh = []
                for cnt1 in bluecnts2[cnt]:
                # ___________________________________________________________________________________ 컨투어 사각형 좌표 리턴
                    _x, _y, _w, _h = np.array(cv2.boundingRect(cnt1) + np.array([-5,-10,10,20])) # add padding to rectangles
                # ___________________________________________________________________________________
                    xywh.append(list(map(int, [_x, _y, _w, _h])))
                # _______________________________________________________________________________________ 중복 사각형 병합
                rect, _ = cv2.groupRectangles(xywh+xywh, 0, 0.001) # merge contours with intersection
                # _______________________________________________________________________________________
                print('o'*135)
                plt.figure(figsize=(30,10))    
                plt.subplot(1,4,1)
                plt.imshow(rect2)
                plt.subplot(1,4,2)
                plt.imshow(rect4)
                plt.subplot(1,4,3)
                plt.imshow(rect)
                plt.subplot(1,4,4)
                plt.imshow(img_resized_2)
                plt.show()
                print('o'*135)

        img_detection = np.concatenate((img_detection, img_resized_2[np.newaxis,...]), axis=0)        
        print(cnt_1)
        temp = string_path[0:-4].split('CAM_')        
#        cv2.imwrite('img//UP_OK_2/{}.jpg'.format(temp[1]), cv2.cvtColor(img_resized_2, cv2.COLOR_RGB2BGR))        
        print(string_path)
#         plt.figure(figsize=(30,10))    
#         plt.subplot(1,4,1)
#         plt.imshow(img_color)
#         plt.subplot(1,4,2)
#         plt.imshow(rect2)
# #         plt.subplot(1,4,3)
# #         plt.imshow(rect3)
#         plt.subplot(1,4,3)
#         plt.imshow(dilation) 
#         plt.subplot(1,4,4)
#         plt.imshow(img_resized_2) 
#         plt.show()
        print('='*135)
        

    return img_detection


# detectiony_50_100

In [ ]:
img_up_ok_det = detection_y(up_ok, path_list_up_ok)

In [ ]:
img_up_ng_det = detection_y(up_ng, path_list_up_ng)

In [ ]:
img_up_ok_det = detection_y_1(up_ok, path_list_up_ok)

In [ ]:
img_up_ng_det = detection_y_1(up_ng, path_list_up_ng)

In [ ]:
img_up_ok_det.shape, img_up_ng_det.shape

In [ ]:
X = np.concatenate((img_up_ng_det, img_up_ok_det))
y = np.concatenate((np.ones(len(img_up_ng_det)), np.zeros(len(img_up_ok_det))))

In [ ]:
X.shape, y.shape

In [ ]:
# grayscale
#X = X[:,:,:,-1]
#X.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=77)

In [ ]:
from keras import optimizers, metrics
from keras.layers import Input, Dense, Conv2D, Activation, GlobalAveragePooling2D, Flatten, concatenate
from keras.layers import Dropout, BatchNormalization, MaxPooling2D, Lambda
from keras.models import Model, Input
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
#from keras.applications.mobilenetv2 import MobileNetV2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import multi_gpu_model


In [ ]:
from keras.utils.vis_utils import plot_model
import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 48
        self.img_cols = 100
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        #optimizer = Adam(0.0008, 0.5)
        optimizer = RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)
        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        
        # Generator의 결과는 이미지 입니다.
        img = self.generator(z)

        # For the combined model we will only train the generator
        # Generator와 Discriminator를 동시에 학습시키고 싶을 때 trainable을 False로 설정
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity        
        # Discriminator의 결과는 이미지가 진짜인지 가짜인지에 대한 확률입니다.
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)
        self.combined = Model(z, valid)
        self.combined = multi_gpu_model(self.combined, gpus=2)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


        
        
    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 12 * 25 , activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((12, 25, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        
        #model.add(Flatten())
        model.add(GlobalAveragePooling2D())        
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50, X_train = X_train):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()        
        # Rescale -1 to 1
        #X_train = (X_train / 127.5) - 1.
        X_train = (X_train)/255.0
        #X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        import time
        
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------
            startTime = time.time()
            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            #self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            #self.discriminator.trainable = False 
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0 or epoch == epochs - 1 :    
                cnt = 0
                #gen_imgs = np.uint8(255 * gen_imgs)
                for i in range(0,10):
                    #plt.imshow(gen_imgs[cnt, :,:,0])#, cmap='gray')                    
                    cv2.imwrite('fake_img/fake_{0}_{1}_acc_{2}per.png'.format(epoch,i,100*d_loss[1]),(cv2.cvtColor(np.uint8(255 * gen_imgs[cnt, :,:,:]), cv2.COLOR_RGB2BGR)))
                    #cv2.imwrite('fake_img/fake_{0}_{1}.png'.format(epoch,i),np.uint8(255 * gen_imgs[cnt, :,:,:]))
                    #print(np.uint8(255 * gen_imgs[cnt, :,:,0]))
                    cnt += 1
                #self.save_imgs(epoch)
                self.save_model_weights()

            endTime = time.time() - startTime
            print('- epoch during time : {}sec'.format(round(endTime,4)))
            
    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        #gen_imgs = (0.5 * gen_imgs + 0.5)* 127.5
        gen_imgs = np.uint8(255 * gen_imgs)


        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0])#, cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("fake_img/fake_{}.png".format(epoch))
        plt.close()
        
    def save_model_weights(self): 
        self.generator.save_weights('gan_g_weights.h5') 
        self.discriminator.save_weights('gan_d_weights.h5')
        self.combined.save_weights('gan_combined_weights.h5')




In [ ]:
if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=6000, batch_size=256, save_interval=10,X_train=X_train)

In [ ]:
X_train.shape

In [ ]:
for i in range(1000,1004):    
    plt.imshow((cv2.cvtColor((cv2.cvtColor(X[:,:,:,:][i], cv2.COLOR_RGB2BGR)), cv2.COLOR_RGB2BGR)))
    plt.show()